<a href="https://colab.research.google.com/github/halyusa16/etl-customer-shopping-data/blob/main/etl_invoice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [52]:
from google.oauth2 import service_account
from google.auth.transport.requests import AuthorizedSession
import gspread
import pandas as pd
from google.cloud import bigquery
import pandas_gbq

In [32]:
scope = ['https://www.googleapis.com/auth/spreadsheets']

# adding the credential file to allow gsheet access from google API
credentials = service_account.Credentials.from_service_account_file('/content/gsheet-viewer.json', scopes = scope)

# specify the spreadsheet that we worked on
gc = gspread.Client(auth=credentials)
gc.session = AuthorizedSession(credentials)

In [38]:
# add and open google sheet using the google sheet ID
sheet_key = '1lNDPMgXggHkyGTtJR4RNlCHtVQa2aUYZOF19K2RKTBo'
sheet = gc.open_by_key(sheet_key)

In [43]:
# E for Extract
# access the worksheet
worksheet_name = 'data'
worksheet = sheet.worksheet(worksheet_name)

# store data into sheet_data variable
sheet_data = worksheet.get_all_records()

In [55]:
# store data into pandas
df = pd.DataFrame(sheet_data)

# time for T, transform (just a little, lol)
# column name formatting
df.columns = ['invoice_number',
              'customer_id',
              'customer_gender',
              'customer_age',
              'product_category',
              'product_quantity',
              'total_price',
              'payment_method',
              'invoice_date',
              'shopping_mall_name']

In [56]:
# time for L in ETL, load!
# load the data to BigQuery

# enter Google Cloud project ID
project_id = 'sound-chalice-349712'

# enter BigQuery dataset and create a table name
dataset_name = 'halyusa_data'
table_name = 'customer_shopping_data'

# construct the full table ID
table_id = f'{project_id}.{dataset_name}.{table_name}'

# push data to BQ using to_gbq
pandas_gbq.to_gbq(df, table_id, project_id=project_id, if_exists='replace')

print("Data successfully loaded to BQ")

100%|██████████| 1/1 [00:00<00:00, 4337.44it/s]

Data successfully loaded to BQ
